## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\eddy\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

#卷積組合
classifier.add(Convolution2D(filters=32, kernel_size=(3,3)))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC 
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
50000/50000 [==============================] - 159s 3ms/step - loss: 1.4035 - acc: 0.5049
Epoch 2/10
50000/50000 [==============================] - 161s 3ms/step - loss: 1.0297 - acc: 0.6384
Epoch 3/10
50000/50000 [==============================] - 160s 3ms/step - loss: 0.8807 - acc: 0.6893
Epoch 4/10
50000/50000 [==============================] - 160s 3ms/step - loss: 0.7708 - acc: 0.7307
Epoch 5/10
50000/50000 [==============================] - 162s 3ms/step - loss: 0.6773 - acc: 0.7625
Epoch 6/10
50000/50000 [==============================] - 162s 3ms/step - loss: 0.5823 - acc: 0.7963
Epoch 7/10
50000/50000 [==============================] - 165s 3ms/step - loss: 0.4898 - acc: 0.8282
Epoch 8/10
50000/50000 [==============================] - 167s 3ms/step - loss: 0.4016 - acc: 0.8611
Epoch 9/10
50000/50000 [==============================] - 162s 3ms/step 

C:\Users\eddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
C:\Users\eddy\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)


array([[3.8058645e-09, 6.2832552e-12, 1.8564679e-10, 9.6971227e-08,
        9.9999988e-01, 1.4107359e-09, 1.4695412e-08, 6.9939969e-09,
        1.9723863e-09, 2.0905796e-16]], dtype=float32)